In [1]:
dataRDD = sc.wholeTextFiles("data/xmlfiles/*")

In [3]:
import re
def xml_parsing(fileName):
    
    text_content = fileName.encode('utf-8')

    # Regular expressions to find all the links and text from XML files and storing them in two lists
    TITLE_RE = re.compile(r'<title>\s*(.+)\s*<\/title>')
    LINK_RE = re.compile(r'\[\[([^\]]+)\]')
    TEXT_RE = re.compile(r'<text.+>\s*(.+)\s*<\/text>')

    #print 'Links:' , LINK_RE.findall(text_content)
    #print 'Text: ', TEXT_RE.findall(text_content)
    title = TITLE_RE.findall(text_content)
    #Creating a tuple of links, text where each of them is a list
    tup = (LINK_RE.findall(text_content), TEXT_RE.findall(text_content))

    return " ".join(title), tup


In [4]:
contents = dataRDD.values().map(xml_parsing)
linkRDD = contents.flatMap(lambda x: [x[0]])

In [15]:
resultRDD= linkRDD.zip(dataRDD.map(lambda x : x[0]))
#resultRDD.map(lambda x: (x[0][1],x[0][0],x[1]))
linkList = sorted(resultRDD.collect())
linked={}
for i, j in enumerate(linkList):
    linked[j[1]]= (i, j[0])
# print 'Link is:', link    
import json
with open('PageTitles.json', 'w') as fp:
    json.dump(linked, fp)
    
linkListMain = sorted(linkRDD.collect())
link = {j:i for i, j in enumerate(linkListMain)}

In [16]:
import numpy as np
import operator

nVector = [(0.15/len(link))]*len(link)
def documentTermVector(i, fileLinks):
    vecList = [0.0]*len(link)
    m = 0
    print len(fileLinks)
    for each in fileLinks:
        if "|" in each:
            x = each.split("|")[0]
            try:
                if(link[x]):
                    m += 1
            except KeyError:
                continue
        else:
            try:
                print each
                if(link[each]+1):
                    print m
                    m += 1
            except KeyError:
                continue
    print m
    for each in fileLinks:
        if "|" in each:
            x = each.split("|")[0]
            try:
                vecList[link[x]] += 0.85/m
            except KeyError:
                continue
        else:
            try:
                vecList[link[each]] += 0.85/m
            except KeyError:
                continue
    tempList = map(operator.add, vecList,nVector)
    finallist = [[link[i], s, r] for s, r in enumerate(tempList)]
    return finallist

In [17]:
tdmMatrix = contents.flatMap(lambda x: documentTermVector(x[0], x[1][0]))
colMat = tdmMatrix.map(lambda x: (x[1], [x[2]])).reduceByKey(lambda p,q:p+q)

In [18]:
randomSurfer = sc.parallelize([(1.0/len(link))]*len(link)).map(lambda x: (0, [x])).reduceByKey(lambda p,q:p+q)
#randomSurfer.collect()
# tdmMatrix.take(2)
# documentTermVector(1, ['Amuck',
#                         'Amuck',                                           
# 'Amuck','Amuck',
#  'Symphony No. 9 (Beethoven)',
#  'Running amok',
#  'BiblicalInterpretation',
#  'Abiword',
#  'BirthofaNation',
#  'RUR-5 ASROC',
#  'Alexandria Troas',
#  'Benjamin Lee Whorf',
#  'Amnon'])

In [19]:
cartRDD = colMat.cartesian(randomSurfer)\
                .map(lambda x: np.dot(x[0][1], x[1][1]))

In [20]:
cart2RDD = colMat.cartesian(cartRDD.map(lambda x: (0, [x])).reduceByKey(lambda p,q:p+q))\
                .map(lambda x: np.dot(x[0][1], x[1][1]))

In [21]:
# normRdd = cartRDD.map(lambda x: (0, [x]))\
#                  .reduceByKey(lambda p,q:p+q)\
#                  .cartesian(cart2RDD.map(lambda x: (0, [x])).reduceByKey(lambda p,q:p+q))\
#                  .map(lambda x: np.linalg.norm((np.array(x[0][1]) - np.array(x[1][1])), ord = 2))

# normRdd.collect()    

In [22]:
pageRankDict = {}
def pageRank(vprime, vnewprime, c):
    c+=1
    norm = []
    norm = vprime.map(lambda x: (0, [x]))\
                 .reduceByKey(lambda p,q:p+q)\
                 .cartesian(vnewprime.map(lambda x: (0, [x])).reduceByKey(lambda p,q:p+q))\
                 .map(lambda x: np.linalg.norm((np.array(x[0][1]) - np.array(x[1][1])), ord = 2)).collect()
    n = float(norm[0])
    print "Norm is: ", c, n, type(n)
    if n <= 0.000001:
        pageRankList = vnewprime.collect()
        for i, j in enumerate(pageRankList):
            pageRankDict[i]=j
        with open('PageRankResults.json', 'w' ) as fp:
            json.dump(pageRankDict, fp)
        return pageRankList
    else:
        vNew = colMat.cartesian(vnewprime.map(lambda x: (0, [x]))\
                     .reduceByKey(lambda p,q:p+q))\
                     .map(lambda x: np.dot(x[0][1], x[1][1]))
        pageRank(vnewprime, vNew, c)

In [23]:
pg = pageRank(cartRDD, cart2RDD, 0)

Norm is:  1 0.0290058577432 <type 'float'>
Norm is:  2 0.0136369135139 <type 'float'>
Norm is:  3 0.00868227558324 <type 'float'>
Norm is:  4 0.00344827039967 <type 'float'>
Norm is:  5 0.00199242046689 <type 'float'>
Norm is:  6 0.00125518706068 <type 'float'>
Norm is:  7 0.000763624222336 <type 'float'>
Norm is:  8 0.000475836575444 <type 'float'>
Norm is:  9 0.000290376885166 <type 'float'>
Norm is:  10 0.000180455621675 <type 'float'>
Norm is:  11 0.000110318041291 <type 'float'>
Norm is:  12 6.8450449741e-05 <type 'float'>
Norm is:  13 4.18745274133e-05 <type 'float'>
Norm is:  14 2.59670352321e-05 <type 'float'>
Norm is:  15 1.58928053764e-05 <type 'float'>
Norm is:  16 9.85212113048e-06 <type 'float'>
Norm is:  17 6.03226584475e-06 <type 'float'>
Norm is:  18 3.73853247895e-06 <type 'float'>
Norm is:  19 2.28986661633e-06 <type 'float'>
Norm is:  20 1.41884083711e-06 <type 'float'>
Norm is:  21 8.6934346973e-07 <type 'float'>


In [24]:
link["Brazil"]

205

In [25]:
from pyspark.mllib.feature import HashingTF

In [26]:
def text_parsing(fileName):
    text_content = fileName.encode('utf-8')

    # Regular expressions to find all the links and text from XML files and storing them in two lists
    TEXT_RE = re.compile(r'<text.+>([\s\S]*)<\/text>')
    
    liste = TEXT_RE.findall(text_content)
    str1 = re.split('[^a-zA-Z.]', liste[0].lower())
    str2 = filter (None, str1)
    return str2

splitRDD = dataRDD.values().map(text_parsing)

In [27]:
hashingTF = HashingTF()
tf = hashingTF.transform(splitRDD)
from pyspark.mllib.feature import IDF

# ...from tf create IDF
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [28]:
zipped = splitRDD.zip(tfidf)
fRDD = splitRDD.flatMap(lambda x: x).distinct()
print fRDD.count()

63556


In [29]:
wordRDD = fRDD.map(lambda x: (x, hashingTF.indexOf(x)))
listW = wordRDD.collect()

In [30]:
import json
dictW = dict(listW)

with open("wordHashD.json", 'w') as f:
    json.dump(dictW, f)

In [31]:
finalZip = dataRDD.map(lambda x: x[0]).zip(zipped)
# finalZip.take(2)

In [51]:
def searchWords(words):
    words = words.lower()
    resultList = []
    for word in words.split(" "):
        resultList.append(finalZip.map(lambda x: (x[0], x[1][0], x[1][1]))\
                     .filter(lambda x: word in x[1])\
                     .map(lambda x: (x[2][dictW[word]], x[0]))\
                     .map(lambda x : (x[1], pageRankDict[linked[x[1]][0]], x[0], linked[x[1]][1]))\
                     .sortBy(lambda x: x[1], False)\
                     .sortBy(lambda x: x[2], False).take(5))
    return resultList

In [55]:
searchWords("Black Forest")

[[(u'file:/home/vagrant/data/xmlfiles/page-0001961.xml',
   3.6173057647550886e-09,
   368.05060426715016,
   'Black Forest'),
  (u'file:/home/vagrant/data/xmlfiles/page-0001962.xml',
   3.4946785748791316e-08,
   278.60404307075112,
   'Black Sea'),
  (u'file:/home/vagrant/data/xmlfiles/page-0001818.xml',
   8.6398269018719571e-09,
   30.793078444661965,
   'Atlanta'),
  (u'file:/home/vagrant/data/xmlfiles/page-0001928.xml',
   8.6398269018719571e-09,
   29.32674137586854,
   'The Birth of a Nation'),
  (u'file:/home/vagrant/data/xmlfiles/page-0001781.xml',
   1.1477225206606812e-08,
   24.927730169488257,
   'Alexandria')],
 [(u'file:/home/vagrant/data/xmlfiles/page-0001961.xml',
   3.6173057647550886e-09,
   654.0620861526919,
   'Black Forest'),
  (u'file:/home/vagrant/data/xmlfiles/page-0001783.xml',
   3.6173057647550886e-09,
   23.08454421715383,
   'Alexandria, Louisiana'),
  (u'file:/home/vagrant/data/xmlfiles/page-0001960.xml',
   7.3685201693453462e-07,
   17.954645502230758